Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
#DAF
from sklearn.linear_model import LogisticRegression

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [ ]:
(samples, width, height) = train_dataset.shape
train_X = np.reshape(train_dataset,(samples,width*height))
train_y = train_labels

(samples, width, height) = test_dataset.shape
test_X = np.reshape(test_dataset,(samples,width*height))
test_y = test_labels

N=10000
N_test=10000
C=0.001

model_l2 = LogisticRegression(multi_class='multinomial', solver='lbfgs', C=C, penalty='l2', tol=0.01)
model_l2.fit(train_X[0:N], train_y[0:N])
model_l2_coef = model_l2.coef_.ravel()

y_hat = model_l2.predict(test_X[0:N_test])
error = (1 - np.mean(y_hat == test_y[0:N_test]))
accuracy = sum(y_hat == test_y[0:N_test]) / float(N_test)

print("N=%s, N_test=%s, l2_penalty_coeff=%.2f, C=%.4f" % (N,N_test,1.0/C,C))
print("Score with L2 penalty: %.4f" % model_l2.score(train_X[0:N], train_y[0:N]))
print("Test Accuracy Rate: %.4f AND Test Error Rate: %.4f\n" % (accuracy, error))

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape, )
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [ ]:
#DAF understanding
print(train_dataset[0])
print(train_labels[0])
#

import matplotlib.pyplot as plt
%matplotlib inline
first = np.reshape(train_dataset[0],(28,28))
print(first.shape)
plt.imshow(first[:,:])
plt.show()

We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [ ]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable( tf.truncated_normal([image_size * image_size, num_labels]) )
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)  
    
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

  #DAF understanding
  print(tf_train_dataset)
  print(tf_train_labels)
  print(tf_valid_dataset)
  print(tf_test_dataset)
  print(weights)
  print(biases)
  print(logits)
  print(loss)
  print(train_prediction)
  print(valid_prediction)
  print(test_prediction)
  #

Let's run this computation and iterate:

In [ ]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [6]:
batch_size = 128

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.921280
Minibatch accuracy: 13.3%
Validation accuracy: 15.9%
Minibatch loss at step 500: 1.735652
Minibatch accuracy: 70.3%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 1.196321
Minibatch accuracy: 77.3%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 0.953729
Minibatch accuracy: 78.1%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 1.030213
Minibatch accuracy: 76.6%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 0.812614
Minibatch accuracy: 78.9%
Validation accuracy: 77.9%
Minibatch loss at step 3000: 0.803204
Minibatch accuracy: 81.2%
Validation accuracy: 79.2%
Test accuracy: 85.9%


In [ ]:
#DAF understanding: offset....
batch_size = 128
#num_steps = 3001
num_steps = 10

for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    print ("step=[%s] DAF offset:[%s], GOOGLE offset:[%s]" % (step, step * batch_size, offset))
    print ("batch_data = train_dataset[%s:%s, :]" % (offset, offset + batch_size))
    print ("batch_labels = train_labels[%s:%s, :]\n" % (offset, offset + batch_size))    

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [16]:
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    
  #DAF: ATT. THIS IS ALWAYS NECESSARY TO IMPROVE PERFORMANCE: PUT DIFFERENTS W STDDEVS IN EACH HIDDEN LAYER
  #weights1 = tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.1))
  #weights = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels],  stddev=0.11))
    
  weights = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

  logits = tf.matmul(hidden1, weights) + biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  ## Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights) 
                                   + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights) 
                                  + biases)

In [17]:
import time

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      time.sleep( 30 )
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 320.179657
Minibatch accuracy: 8.6%
Validation accuracy: 29.7%
Minibatch loss at step 500: 14.492945
Minibatch accuracy: 80.5%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 9.276477
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 3.566716
Minibatch accuracy: 82.0%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 4.804980
Minibatch accuracy: 81.2%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 9.827881
Minibatch accuracy: 82.8%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 2.266851
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Test accuracy: 88.2%


In [10]:
#DAF Understanding, How to print Tensorflow Vars.
graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.constant(train_dataset[:1, :])
  tf_train_labels = tf.constant(train_labels[:1])

  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024])) 
  vmatmul = tf.Variable(tf.matmul(tf_train_dataset, weights1) + biases1)
  vhidden1 = tf.Variable(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1))

  print("vmatmul:", vmatmul.shape)
  print("vhidden1:", vhidden1.shape)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()

  print(session.run(vmatmul[0,1:10]))
  print(session.run(vhidden1[0,1:10]))

vmatmul: (1, 1024)
vhidden1: (1, 1024)
[ -0.23964709  13.27432251  12.96650505  16.90345573  -8.56540489
  -1.9062655  -10.72524834  15.87216377   5.6168375 ]
[  0.          13.27432251  12.96650505  16.90345573   0.           0.           0.
  15.87216377   5.6168375 ]


In [23]:
import math
batch_size = 128

# helper functions
def weight_variable(shape, stddev=0.1):
  initial = tf.truncated_normal(shape, stddev=stddev)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layers...  
    hidden_layer1_size = 1024
    hidden_layer2_size = 1024
    hidden_layer3_size = 1024
    
    stddev=math.sqrt(3.0/((image_size * image_size )+ hidden_layer1_size))
    
    # variables
    hidden1_weights = weight_variable([image_size * image_size, hidden_layer1_size], stddev)
    hidden1_biases= bias_variable([hidden_layer1_size])
    hidden1_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)
    
    # variables for 2nd layer
    hidden2_weights = weight_variable([hidden_layer1_size, hidden_layer2_size ], stddev)
    hidden2_biases = bias_variable([hidden_layer2_size])
    hidden2_layer = tf.nn.relu(tf.matmul(hidden1_layer, hidden2_weights) + hidden2_biases)        
    
    # variables for 3nd layer
    hidden3_weights = weight_variable([hidden_layer2_size, hidden_layer3_size ], stddev)
    hidden3_biases = bias_variable([hidden_layer3_size])
    hidden3_layer = tf.nn.relu(tf.matmul(hidden2_layer, hidden3_weights) + hidden3_biases)        
    
    
    output_weights = weight_variable([1024, num_labels])
    output_biases = bias_variable([num_labels])
    
    #trianing computation
    logits = tf.matmul(hidden3_layer, output_weights) + output_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) 

    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    #predcitions
    train_prediction = tf.nn.softmax(logits)

    # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, hidden2_weights) + hidden2_biases)        
    valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, hidden3_weights) + hidden3_biases)        
    valid_logits = tf.matmul(valid_hidden3, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

    # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, hidden2_weights) + hidden2_biases) 
    test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, hidden3_weights) + hidden3_biases) 
    test_logits = tf.matmul(test_hidden3, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)
    
import time

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

num_steps = 3001
#num_steps = 6002

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      time.sleep( 30 )
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.690682
Minibatch accuracy: 7.0%
Validation accuracy: 24.0%
Minibatch loss at step 500: 0.427825
Minibatch accuracy: 90.6%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.300279
Minibatch accuracy: 93.0%
Validation accuracy: 87.6%
Minibatch loss at step 1500: 0.350985
Minibatch accuracy: 88.3%
Validation accuracy: 88.6%
Minibatch loss at step 2000: 0.349478
Minibatch accuracy: 89.1%
Validation accuracy: 88.6%
Minibatch loss at step 2500: 0.284659
Minibatch accuracy: 89.8%
Validation accuracy: 89.3%
Minibatch loss at step 3000: 0.335176
Minibatch accuracy: 91.4%
Validation accuracy: 89.3%
Test accuracy: 94.7%
